In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Merging dataframes

First import all the different csv files as dataframes

In [2]:
auctions = pd.read_csv('tba_auctions_3_years.csv')
bids = pd.read_csv('tba_bids_3_years.csv')
buyer_set = pd.read_csv('tba_buyer_settings_3_years.csv')
buyers = pd.read_csv('tba_buyers_3_years.csv')
category = pd.read_csv('tba_category_3_years.csv')
lots = pd.read_csv('tba_lots_3_years.csv')

C:\Users\20192513\AppData\Local\Temp\ipykernel_1908\163414716.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  lots = pd.read_csv('tba_lots_3_years.csv')


Create a big dataframe where the lots and auction dataframe are combined

In [3]:
merge_cat = pd.merge(category,lots,on='category_id',how='inner')
lots_auc = pd.merge(merge_cat,auctions,on='auction_id',how='inner')

Create a big dataframe for the bids and the buyer information

In [4]:
merge_buy = pd.merge(buyer_set,buyers,on='fake_buyer_id',how='inner')
buy_bid = pd.merge(merge_buy,bids,on='fake_buyer_id',how='inner')

This results in two big dataframes: lots_auc and buy_bid

In [5]:
lots_auc.info()
len(lots_auc)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5007339 entries, 0 to 5007338
Data columns (total 30 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   category_id               int64  
 1   category_level_1          object 
 2   category_level_2          object 
 3   category_level_3          object 
 4   lot_id                    object 
 5   auction_id                float64
 6   auction_start_time        object 
 7   auction_close_time        object 
 8   lot_close_time_actual     object 
 9   estimated_price           float64
 10  starting_price            float64
 11  is_delivery               int64  
 12  auction_country           object 
 13  lot_quantity              float64
 14  lot_title                 object 
 15  lot_description           object 
 16  appearance                object 
 17  condition                 object 
 18  packaging                 object 
 19  assigned_explicitly       object 
 20  is_offered              

5007339

In [6]:
buy_bid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25265755 entries, 0 to 25265754
Data columns (total 19 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   notify_by_email                   object 
 1   notify_by_push                    object 
 2   notify_by_sms                     object 
 3   enable_one_click_bidding          bool   
 4   enable_one_click_reverse_bidding  bool   
 5   open_close_notifications          bool   
 6   open_close_notification_time      int64  
 7   subscribe_on_newsletter           bool   
 8   fake_buyer_id                     int64  
 9   is_company                        bool   
 10  created_at                        object 
 11  updated_at                        object 
 12  is_blocked                        bool   
 13  country                           object 
 14  lot_id                            object 
 15  id                                object 
 16  amount_in_euro                    

# Cleaning Data

We start with cleaning the data of the Lots_auc dataframe. This dataframe seems most important for our research topic: Auction performance.

First remove all the rows that have a NaN value for important columns

In [7]:
lots_auc = lots_auc.dropna(subset=['lot_quantity', 'estimated_price', 'nr_bids','highest_valid_bid_amount','nr_valid_bid_users','is_sold'])

Since we have a dataset considering the data of 3-years we decided to delete the lots with an estimated price of 0 or less than 0. 

In [8]:
lots_auc.drop(lots_auc[lots_auc['estimated_price']<=0].index, inplace=True)

Than we delete all the lots where the starting price is higher than the estimated price

In [9]:
lots_auc = lots_auc[lots_auc['starting_price']<=lots_auc['estimated_price']]

Than we filtered the dataframe to discard data above the 95th percentile of lot quantity

In [14]:
percentile_95 = lots_auc['lot_quantity'].quantile(0.95)
lots_auc = lots_auc[lots_auc['lot_quantity']<= percentile_95]